In [1]:
import torch

In [2]:
torch.cuda.device_count()

2

In [1]:
import sys
import os
sys.path.append(os.path.join(os.getcwd() , '..'))  # Parent directory where is the fedstellar module

In [2]:
import threading

In [5]:
lock = threading.Lock()

In [6]:
import time
from datetime import datetime
from pytorch_lightning.demos.mnist_datamodule import MNISTDataModule
from fedstellar.node import Node
from fedstellar.learning.pytorch.datamodule import DataModule
from fedstellar.learning.pytorch.mnist.mnist import MNISTDataset
from fedstellar.learning.pytorch.syscall.syscall import SYSCALLDataset
from fedstellar.learning.pytorch.cifar10.cifar10 import CIFAR10Dataset
from fedstellar.learning.pytorch.fashionmnist.fashionmnist import FashionMNISTDataset
from fedstellar.config.config import Config
from fedstellar.learning.pytorch.mnist.models.mlp import MNISTModelMLP
from fedstellar.learning.pytorch.mnist.models.cnn import MNISTModelCNN
from fedstellar.learning.pytorch.fashionmnist.models.mlp import FashionMNISTModelMLP
from fedstellar.learning.pytorch.fashionmnist.models.cnn import FashionMNISTModelCNN
from fedstellar.learning.pytorch.sentiment140.models.cnn import SentimentModelCNN
from fedstellar.learning.pytorch.sentiment140.models.rnn import SentimentModelRNN
from fedstellar.learning.pytorch.cifar100.models.cnn import Cifar100ModelCNN
from fedstellar.learning.pytorch.cifar10.models.cnn import Cifar10ModelCNN
from fedstellar.learning.pytorch.syscall.models.mlp import SyscallModelMLP
from fedstellar.learning.pytorch.syscall.models.autoencoder import SyscallModelAutoencoder
from fedstellar.learning.pytorch.cifar10.models.resnet import CIFAR10ModelResNet
from fedstellar.learning.pytorch.cifar10.models.fastermobilenet import FasterMobileNet
from fedstellar.learning.pytorch.cifar10.models.simplemobilenet import SimpleMobileNetV1
from fedstellar.node import Node

In [7]:
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
root = os.path.dirname(os.path.join(os.getcwd()))
single_node = os.path.join(root, "app", "single_node")
print(single_node)

config_path =  os.path.join(single_node, "participant_0.json")
config = Config(entity="participant", participant_config_file=config_path)

d:\git\FedStellar-topo-manipulation\app\single_node


In [8]:
host = "127.0.0.1"
idx = 0
n_nodes = 10
indices_dir = single_node
is_iid = True

In [9]:
label_flipping = False
data_poisoning = False
model_poisoning = False
poisoned_percent = 0
poisoned_ratio = 0
targeted = False
target_label = 0
target_changed_label = 0
noise_type = "salt"
port = 45602
benign_mnist_dataset = CIFAR10Dataset(iid=is_iid)
benign_mnist_model = SimpleMobileNetV1()

benign_mnist_dataset = DataModule(benign_mnist_dataset.train_set, benign_mnist_dataset.test_set, sub_id=idx, number_sub=n_nodes, indices_dir=indices_dir,
                        label_flipping=label_flipping, data_poisoning=data_poisoning,
                        poisoned_percent=poisoned_percent,
                        poisoned_ratio=poisoned_ratio, targeted=targeted, target_label=target_label,
                        target_changed_label=target_changed_label, noise_type=noise_type)

Files already downloaded and verified
Files already downloaded and verified
datamodule: targeted is False
Datapoisoning: generating watermarked samples for testing (all classes)
Train: 4500 Val:500 Test:1000


Config for benign node - MNIST

In [10]:
benign_mnist_node = Node(
        idx=0,
        experiment_name="Single_Node_Test",
        model=benign_mnist_model,
        data=benign_mnist_dataset,
        hostdemo=True,
        host=host,
        port=port,
        config=config,
        encrypt=False,
        model_poisoning=model_poisoning,
        poisoned_ratio=poisoned_ratio,
        noise_type=noise_type,
    )
benign_mnist_node.start()
time.sleep(20)

[BASENODE] Trying to bind to 127.0.0.1:45602


In [19]:
benign_mnist_node.set_start_learning(rounds=1, epochs=10)

In [14]:
benign_mnist_node._start_time -  benign_mnist_node._current_time

-3.1306302547454834

In [17]:
from fedstellar.utils.observer import Events, Observer

In [22]:
benign_mnist_node.update(Events.STOP_LEARNING_EVENT,"s")

In [21]:
benign_mnist_node

<Node(node-127.0.0.1:45602, started 5832)>

In [16]:
while True:
        time.sleep(1)
        finish = True
        for f in [n.round is None for n in [benign_mnist_node]]:
            finish = finish and f

        if finish:
            break

for n in [benign_mnist_node]:
    n.stop()

Output()

Epoch end: Test, epoch number: 0

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       Test/Accuracy       │    0.09999997168779373    │
│       Test/F1Score        │   0.016182301566004753    │
│         Test/Loss         │    2.3036112785339355     │
│      Test/Precision       │   0.008971774019300938    │
│        Test/Recall        │    0.09999997168779373    │
│    TestEpoch/Accuracy     │    0.10000000149011612    │
│     TestEpoch/F1Score     │   0.016466233879327774    │
│    TestEpoch/Precision    │   0.008971774019300938    │
│     TestEpoch/Recall      │    0.10000000149011612    │
└───────────────────────────┴───────────────────────────┘

┏━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃   ┃ Name          ┃ Type                      ┃ Params ┃       In sizes ┃      Out sizes ┃
┡━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ 0 │ train_metrics │ MetricCollection          │      0 │              ? │              ? │
│ 1 │ val_metrics   │ MetricCollection          │      0 │              ? │              ? │
│ 2 │ test_metrics  │ MetricCollection          │      0 │              ? │              ? │
│ 3 │ cm            │ MulticlassConfusionMatrix │      0 │              ? │              ? │
│ 4 │ criterion     │ CrossEntropyLoss          │      0 │              ? │              ? │
│ 5 │ model         │ Sequential                │  134 K │ [1, 3, 32, 32] │ [1, 256, 1, 1] │
│ 6 │ fc            │ Linear                    │  2.6 K │       [1, 256] │        [1, 10] │
└───┴───────────────┴───────────────────────────┴────────┴────────────────┴────────────────┘

Trainable params: 136 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 136 K                                                                                                
Total estimated model params size (MB): 0

...

Epoch end: Validation, epoch number: 2

Epoch end: Train, epoch number: 1

KeyboardInterrupt: 

Epoch end: Validation, epoch number: 3

Epoch end: Train, epoch number: 2

In [6]:
benign_mnist_node = Node(
        idx=0,
        experiment_name="Single_Node_Test",
        model=benign_mnist_model,
        data=benign_mnist_dataset,
        hostdemo=True,
        host=host,
        port=port,
        config=config,
        encrypt=False,
        model_poisoning=model_poisoning,
        poisoned_ratio=poisoned_ratio,
        noise_type=noise_type,
    )
benign_mnist_node.start()
time.sleep(5)
benign_mnist_node.set_start_learning(rounds=1, epochs=10)
while True:
        time.sleep(1)
        finish = True
        for f in [n.round is None for n in [benign_mnist_node]]:
            finish = finish and f

        if finish:
            break

for n in [benign_mnist_node]:
    n.stop()

[BASENODE] Trying to bind to 127.0.0.1:45602


KeyboardInterrupt: 

In [24]:
label_flipping = False
data_poisoning = False
model_poisoning = False
poisoned_percent = 0
poisoned_ratio = 0
targeted = False
target_label = 0
target_changed_label = 0
noise_type = "salt"
port = 45602
benign_mnist_dataset = MNISTDataset(iid=is_iid)
benign_mnist_model = MNISTModelMLP()

benign_mnist_dataset = DataModule(benign_mnist_dataset.train_set, benign_mnist_dataset.test_set, sub_id=idx, number_sub=n_nodes, indices_dir=indices_dir,
                        label_flipping=label_flipping, data_poisoning=data_poisoning,
                        poisoned_percent=poisoned_percent,
                        poisoned_ratio=poisoned_ratio, targeted=targeted, target_label=target_label,
                        target_changed_label=target_changed_label, noise_type=noise_type)


datamodule: targeted is False
Datapoisoning: generating watermarked samples for testing (all classes)
Train: 5400 Val:600 Test:1000


d:\git\FedStellar-topo-manipulation\fedstellar\..\fedstellar\attacks\poisoning\datapoison.py:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(img).clone().detach()


In [20]:
benign_mnist_node = Node(
        idx=0,
        experiment_name="Single_Node_Test",
        model=benign_mnist_model,
        data=benign_mnist_dataset,
        hostdemo=True,
        host=host,
        port=port,
        config=config,
        encrypt=False,
        model_poisoning=model_poisoning,
        poisoned_ratio=poisoned_ratio,
        noise_type=noise_type,
    )
benign_mnist_node.start()
time.sleep(5)
benign_mnist_node.set_start_learning(rounds=1, epochs=10)
while True:
        time.sleep(1)
        finish = True
        for f in [n.round is None for n in [benign_mnist_node]]:
            finish = finish and f

        if finish:
            break

for n in [benign_mnist_node]:
    n.stop()

Output()

Epoch end: Test, epoch number: 0

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       Test/Accuracy       │    0.09000767767429352    │
│       Test/F1Score        │    0.04667884483933449    │
│         Test/Loss         │    2.2997682094573975     │
│      Test/Precision       │   0.038008131086826324    │
│        Test/Recall        │    0.09000767767429352    │
│    TestEpoch/Accuracy     │    0.08722295612096786    │
│     TestEpoch/F1Score     │    0.04937143623828888    │
│    TestEpoch/Precision    │    0.04641176760196686    │
│     TestEpoch/Recall      │    0.08722295612096786    │
└───────────────────────────┴───────────────────────────┘

┏━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┓
┃   ┃ Name          ┃ Type                      ┃ Params ┃ In sizes ┃ Out sizes ┃
┡━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━┩
│ 0 │ train_metrics │ MetricCollection          │      0 │        ? │         ? │
│ 1 │ val_metrics   │ MetricCollection          │      0 │        ? │         ? │
│ 2 │ test_metrics  │ MetricCollection          │      0 │        ? │         ? │
│ 3 │ cm            │ MulticlassConfusionMatrix │      0 │        ? │         ? │
│ 4 │ criterion     │ CrossEntropyLoss          │      0 │        ? │         ? │
│ 5 │ l1            │ Linear                    │  200 K │ [1, 784] │  [1, 256] │
│ 6 │ l2            │ Linear                    │ 32.9 K │ [1, 256] │  [1, 128] │
│ 7 │ l3            │ Linear                    │  1.3 K │ [1, 128] │   [1, 10] │
└───┴───────────────┴───────────────────────────┴────────┴──────────┴───────────┘

Trainable params: 235 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 235 K                                                                                                
Total estimated model params size (MB): 0

...

Epoch end: Train, epoch number: 9

Output()

Epoch end: Test, epoch number: 1

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       Test/Accuracy       │    0.9026536345481873     │
│       Test/F1Score        │    0.8922603726387024     │
│         Test/Loss         │    0.2664109766483307     │
│      Test/Precision       │    0.9067742228507996     │
│        Test/Recall        │    0.9026536345481873     │
│    TestEpoch/Accuracy     │    0.9199330806732178     │
│     TestEpoch/F1Score     │    0.9202542304992676     │
│    TestEpoch/Precision    │    0.9221217632293701     │
│     TestEpoch/Recall      │    0.9199330806732178     │
└───────────────────────────┴───────────────────────────┘

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\luke-\AppData\Local\Programs\Python\Python311\Lib\logging\__init__.py", line 1113, in emit
    stream.write(msg + self.terminator)
UnicodeEncodeError: 'gbk' codec can't encode character '\u2022' in position 769: illegal multibyte sequence
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "d:\git\FedStellar-topo-manipulation\venv\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "d:\git\FedStellar-topo-manipulation\venv\Lib\site-packages\traitlets\config\application.py", line 1046, in launch_instance
    app.start()
  File "d:\git\FedStellar-topo-manipulation\venv\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_loop.start()
  File "d:\git\FedStellar-topo-manipulation\venv\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start
    self.asyncio_loop.run_for

Config for labelflipping node - MNIST

In [6]:
label_flipping = True
data_poisoning = False
model_poisoning = False
poisoned_percent = 100
poisoned_ratio = 100
targeted = False
target_label = 0
target_changed_label = 0
noise_type = "salt"
port = 45603
label_flipping_mnist_dataset = MNISTDataset(iid=is_iid)
label_flipping_mnist_model = MNISTModelMLP()

label_flipping_mnist_dataset = DataModule(label_flipping_mnist_dataset.train_set, label_flipping_mnist_dataset.test_set, sub_id=idx, number_sub=n_nodes, indices_dir=indices_dir,
                        label_flipping=label_flipping, data_poisoning=data_poisoning,
                        poisoned_percent=poisoned_percent,
                        poisoned_ratio=poisoned_ratio, targeted=targeted, target_label=target_label,
                        target_changed_label=target_changed_label, noise_type=noise_type)

datamodule: targeted is False
[labelFlipping]: Untargeted label-flipping!
[labelFlipping]: 600000 number of labels will be flipped!
Datapoisoning: generating watermarked samples for testing (all classes)
Train: 5400 Val:600 Test:1000


d:\git\FedStellar-topo-manipulation\fedstellar\..\fedstellar\attacks\poisoning\labelflipping.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(new_dataset.targets).detach().clone()
d:\git\FedStellar-topo-manipulation\fedstellar\..\fedstellar\attacks\poisoning\labelflipping.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  class_list = set(torch.tensor(targets).tolist())
d:\git\FedStellar-topo-manipulation\fedstellar\..\fedstellar\attacks\poisoning\datapoison.py:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
 

In [7]:
label_flipping_mnist_node = Node(
        idx=0,
        experiment_name="Single_Node_Test",
        model=label_flipping_mnist_model,
        data=label_flipping_mnist_dataset,
        hostdemo=True,
        host=host,
        port=port,
        config=config,
        encrypt=False,
        model_poisoning=model_poisoning,
        poisoned_ratio=poisoned_ratio,
        noise_type=noise_type,
    )
label_flipping_mnist_node.start()
time.sleep(5)
label_flipping_mnist_node.set_start_learning(rounds=1, epochs=10)
while True:
        time.sleep(1)
        finish = True
        for f in [n.round is None for n in [label_flipping_mnist_node]]:
            finish = finish and f

        if finish:
            break

for n in [label_flipping_mnist_node]:
    n.stop()

[BASENODE] Trying to bind to 127.0.0.1:45603


Output()

Epoch end: Test, epoch number: 0

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       Test/Accuracy       │    0.0901612862944603     │
│       Test/F1Score        │   0.016637280583381653    │
│         Test/Loss         │    2.3033435344696045     │
│      Test/Precision       │   0.010722099803388119    │
│        Test/Recall        │    0.0901612862944603     │
│    TestEpoch/Accuracy     │    0.09223060309886932    │
│     TestEpoch/F1Score     │   0.017755862325429916    │
│    TestEpoch/Precision    │   0.015198711305856705    │
│     TestEpoch/Recall      │    0.09223060309886932    │
└───────────────────────────┴───────────────────────────┘

┏━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┓
┃   ┃ Name          ┃ Type                      ┃ Params ┃ In sizes ┃ Out sizes ┃
┡━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━┩
│ 0 │ train_metrics │ MetricCollection          │      0 │        ? │         ? │
│ 1 │ val_metrics   │ MetricCollection          │      0 │        ? │         ? │
│ 2 │ test_metrics  │ MetricCollection          │      0 │        ? │         ? │
│ 3 │ cm            │ MulticlassConfusionMatrix │      0 │        ? │         ? │
│ 4 │ criterion     │ CrossEntropyLoss          │      0 │        ? │         ? │
│ 5 │ l1            │ Linear                    │  200 K │ [1, 784] │  [1, 256] │
│ 6 │ l2            │ Linear                    │ 32.9 K │ [1, 256] │  [1, 128] │
│ 7 │ l3            │ Linear                    │  1.3 K │ [1, 128] │   [1, 10] │
└───┴───────────────┴───────────────────────────┴────────┴──────────┴───────────┘

...

Epoch end: Train, epoch number: 9

Output()

Epoch end: Test, epoch number: 1

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       Test/Accuracy       │    0.9263056516647339     │
│       Test/F1Score        │    0.9203307628631592     │
│         Test/Loss         │    0.22175036370754242    │
│      Test/Precision       │    0.9299616813659668     │
│        Test/Recall        │    0.9263056516647339     │
│    TestEpoch/Accuracy     │    0.9439323544502258     │
│     TestEpoch/F1Score     │     0.944196343421936     │
│    TestEpoch/Precision    │    0.9450737237930298     │
│     TestEpoch/Recall      │    0.9439323544502258     │
└───────────────────────────┴───────────────────────────┘

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\luke-\AppData\Local\Programs\Python\Python311\Lib\logging\__init__.py", line 1113, in emit
    stream.write(msg + self.terminator)
UnicodeEncodeError: 'gbk' codec can't encode character '\u2022' in position 771: illegal multibyte sequence
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "d:\git\FedStellar-topo-manipulation\venv\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "d:\git\FedStellar-topo-manipulation\venv\Lib\site-packages\traitlets\config\application.py", line 1046, in launch_instance
    app.start()
  File "d:\git\FedStellar-topo-manipulation\venv\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_loop.start()
  File "d:\git\FedStellar-topo-manipulation\venv\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start
    self.asyncio_loop.run_for

In [23]:
from fedstellar.learning.aggregators.helper import cosine_similarity

In [20]:
label_flipping_mnist_dataset.dataset.targets

AttributeError: 'DataModule' object has no attribute 'dataset'

In [22]:
label_flipping_mnist_dataset.train_loader.dataset.dataset.dataset.targets

tensor([5, 0, 4,  ..., 5, 6, 8])

In [34]:
local_params = benign_mnist_node.learner.get_parameters()
model_params = label_flipping_mnist_node.learner.get_parameters()
cosine_similarity(local_params, model_params)

0.08365502208471298

In [36]:
benign_mnist_node.learner.evaluate()

Output()

Epoch end: Test, epoch number: 2

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       Test/Accuracy       │    0.9026536345481873     │
│       Test/F1Score        │    0.8922603726387024     │
│         Test/Loss         │    0.2664109766483307     │
│      Test/Precision       │    0.9067742228507996     │
│        Test/Recall        │    0.9026536345481873     │
│    TestEpoch/Accuracy     │    0.9199330806732178     │
│     TestEpoch/F1Score     │    0.9202542304992676     │
│    TestEpoch/Precision    │    0.9221217632293701     │
│     TestEpoch/Recall      │    0.9199330806732178     │
└───────────────────────────┴───────────────────────────┘

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\luke-\AppData\Local\Programs\Python\Python311\Lib\logging\__init__.py", line 1113, in emit
    stream.write(msg + self.terminator)
UnicodeEncodeError: 'gbk' codec can't encode character '\u2022' in position 771: illegal multibyte sequence
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "d:\git\FedStellar-topo-manipulation\venv\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "d:\git\FedStellar-topo-manipulation\venv\Lib\site-packages\traitlets\config\application.py", line 1046, in launch_instance
    app.start()
  File "d:\git\FedStellar-topo-manipulation\venv\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_loop.start()
  File "d:\git\FedStellar-topo-manipulation\venv\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start
    self.asyncio_loop.run_for

In [3]:
from fedstellar.learning.pytorch.remotelogger import FedstellarWBLogger

In [6]:
import json

In [12]:
os.getcwd()

'd:\\git\\FedStellar-topo-manipulation\\fedstellar'

In [14]:
with open("app/config/CIFAR10_1_MLP_10_DFL_FedAvg_fully_NoAttack_0_N0-S0_R0_salt_20231002_171158/participant_0.json") as f:
    c = json.load(f)

In [18]:
c

{'scenario_args': {'name': 'CIFAR10_1_MLP_10_DFL_FedAvg_fully_NoAttack_0_N0-S0_R0_salt_20231002_085930',
  'start_time': '02/10/2023 08:59:33',
  'federation': 'DFL',
  'rounds': 10,
  'simulation': True,
  'controller': 'host.docker.internal:5000',
  'random_seed': 0,
  'n_nodes': 10,
  'config_version': 'development',
  'debug': True},
 'device_args': {'uid': 'eb06996869a83aa5935e7b22433744ddafc30f6a',
  'idx': 0,
  'username': 'pi',
  'password': 'pi',
  'role': 'aggregator',
  'start': True,
  'accelerator': 'gpu',
  'devices': 1,
  'strategy': 'ddp',
  'logging': True},
 'geo_args': {'latitude': 41.99166377763001, 'longitude': -6.102612152038019},
 'network_args': {'ip': '192.168.50.2',
  'ipdemo': '192.168.50.2',
  'port': 46510,
  'neighbors': '192.168.50.3:46511 192.168.50.4:46512 192.168.50.5:46513 192.168.50.6:46514 192.168.50.7:46515 192.168.50.8:46516 192.168.50.9:46517 192.168.50.10:46518 192.168.50.11:46519',
  'interface': 'eth0',
  'rate': '1Mbps',
  'loss': '0.1%',
  '

In [15]:
project="fedstellar"
group = "test"
name ="p1"
config=c

In [16]:
wandblogger = FedstellarWBLogger(project=project,
                                                 group=group,
                                                 name=name,
                                                 config=config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jifegi. Use `wandb login --relogin` to force relogin


In [20]:
wandblogger._wandb_init

{'name': 'p1',
 'project': 'fedstellar',
 'dir': '.',
 'id': None,
 'resume': 'allow',
 'anonymous': None,
 'config': {'data_args': {'dataset': 'CIFAR10', 'is_iid': True},
  'model_args': {'model': 'MLP'},
  'aggregator_args': {'algorithm': 'FedAvg',
   'sentinel_distance_threshold': 0.1,
   'sentinelglobal_active_round': 3},
  'adversarial_args': {'attacks': 'No Attack',
   'targeted': False,
   'poisoned_sample_percent': 0,
   'poisoned_ratio': 0,
   'target_label': 3,
   'noise_type': 'salt',
   'target_changed_label': 7,
   'attack_env': {'attack': 'No Attack',
    'poisoned_node_percent': 0,
    'poisoned_sample_percent': 0,
    'poisoned_ratio': 0,
    'noise_type': 'salt',
    'targeted': False},
   'poisoned_node_percent': 0,
   'is_iid': True},
  'attack_env': {'attack': 'No Attack',
   'poisoned_node_percent': 0,
   'poisoned_sample_percent': 0,
   'poisoned_ratio': 0,
   'noise_type': 'salt',
   'targeted': False},
  'network_args': '192.168.50.2:46510'},
 'group': 'test'}

: 

In [27]:
0.1/(6000**0.5) + 1/(60000**0.5)

0.005373477353374435

: 